In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [10]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]

In [11]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in week_day_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Apr_list)

60480

In [13]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 2520)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2400)

In [15]:
x[:,:2]

matrix([[ 338826.19914217,  340870.69418863],
        [ 356607.34023537,  358486.38585181],
        [ 637439.82312416,  639230.39632583],
        [ 663692.59872856,  664588.27655425],
        [ 168716.14583333,  169784.36832265],
        [ 230757.04056486,  233410.20644246],
        [ 413870.98604879,  419817.0297148 ],
        [ 418887.708701  ,  410135.43481856],
        [ 397214.3020736 ,  393498.46510653],
        [ 412735.07617971,  420136.97259902],
        [ 390148.43068691,  430028.83789741],
        [ 276985.11125846,  266240.5339992 ],
        [  99100.26568219,  103337.77220933],
        [ 100285.75382569,   95613.56467784],
        [ 157640.81551181,  169261.09059214],
        [ 128118.17574525,  129407.53557805],
        [ 216276.51935404,  216161.04998767],
        [ 218801.92307692,  214551.0093072 ],
        [ 408140.1203505 ,  407087.39149114],
        [ 352353.44050541,  352159.04364636],
        [ 493746.88201778,  498148.97401147],
        [ 578239.81828812,  572200

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e-01, 2e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.68253676e+07  0.00000000e+00  5.44e+03 2.52e+03  1.00e+06     0s
   1  -1.18126299e+07 -7.68717948e+06  9.92e+02 4.60e+02  2.32e+05     0s
   2  -7.64771089e+06 -1.04840479e+07  1.65e+01 7.63e+00  5.58e+04     0s
   3  -8.20544578e+06 -8.50246851e+06  1.65e-05 7.63e-06  5.3

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekday_AM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[2.483384785388589e-09,
 162708.70057401576,
 96779.36663409013,
 1.379715161355403e-06,
 68121.99923733235,
 2.323450999320014e-06,
 2.322550198923662e-06,
 155188.464240378,
 1.5371620503335576e-10,
 134290.3953135892,
 1.536990868145567e-10,
 1.537161991678445e-10,
 1.5370604282447191e-10,
 1.547421537545398e-10,
 1.5910660822117026e-06,
 1.5879409453403042e-06,
 2.7327535077256026e-07,
 1.380149919744686e-06,
 123421.29713381936,
 245431.6728239135,
 2.3016966391255266e-06,
 6.9991367527895854e-06,
 224660.6081660313,
 7.853550280144199e-07,
 0.000949102797471707,
 0.0009968855814330713,
 5194.483560928636,
 82802.0079292785,
 207182.81704192318,
 35315.933423500865,
 1.1396878971826202e-06,
 35946.46413430407,
 104593.61263585293,
 5.490098276145905e-07,
 18597.116420295682,
 13342.55119560248,
 21214.135549526003,
 462052.77381224156,
 1.0699260466897698e-06,
 34256.445752319225,
 468461.6475765155,
 25832.103336461114,
 6.118863190984346e-07,
 6.115703958539876e-07,
 1.438698261